# Fundamental Agent 基本面分析測試

本測試展示：**基本面分析 Agent 的完整工作流程**

使用 5 個自訂基本面分析工具（@function_tool）進行手動測試：
- calculate_financial_ratios
- analyze_financial_health
- evaluate_valuation
- analyze_growth_potential
- generate_investment_rating

## Step 1: 環境設置

In [1]:
import sys
from pathlib import Path
from dotenv import load_dotenv

from agents import (
    Runner,
    gen_trace_id,
    trace,
    WebSearchTool,
    CodeInterpreterTool,
)
from agents.mcp import MCPServerStdio

load_dotenv()

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / 'backend' / 'src'))

from common.logger import logger

print('✓ 環境設置完成')

2025-10-22 09:41:20 | INFO     | common.logger:setup_logger:108 | Logger initialized with level: INFO
2025-10-22 09:41:20 | INFO     | common.logger:setup_logger:110 | Logging to file: /Users/sacahan/Documents/workspace/CasualTrader/backend/logs/backend.log


✓ 環境設置完成


## Step 2: 導入必要的模組

In [2]:
from trading.tools.fundamental_agent import (
    get_fundamental_agent,
)

print('✓ 模組導入完成')

✓ 模組導入完成


## Step 3: 初始化 OpenAI 內建工具

In [3]:
# WebSearchTool: 提供網路搜尋功能
web_search_tool = WebSearchTool(
    user_location=None,  # 可選：用戶位置，用於本地化搜尋結果
    filters=None,  # 可選：搜尋過濾器
    search_context_size="medium",  # 搜尋上下文大小：'low'、'medium'、'high'
)

# CodeInterpreterTool: 提供程式碼執行功能
# 必須指定 type 和 container 設置，container.type 必須為 "auto"
code_interpreter_tool = CodeInterpreterTool(
    tool_config={
        "type": "code_interpreter",
        "container": {
            "type": "auto"  # OpenAI 自動選擇最適合的容器
        },
    }
)

openai_tools = [web_search_tool, code_interpreter_tool]

## Step 4: 初始化 MCP Servers

In [4]:
# 全局變量來管理 MCP servers
casual_market_mcp = None
mcp_memory = None
mcp_servers = []

async def setup_mcp_servers():
    """設置 MCP servers"""
    global casual_market_mcp, mcp_memory, mcp_servers
    
    # 清理舊的連接
    if casual_market_mcp:
        try:
            await casual_market_mcp.__aexit__(None, None, None)
        except:
            pass
    
    if mcp_memory:
        try:
            await mcp_memory.__aexit__(None, None, None)
        except:
            pass
    
    # 初始化 casual-market-mcp
    casual_market_mcp = MCPServerStdio(
        params={
            "command": "uvx",
            "args": [
                "--from",
                "/Users/sacahan/Documents/workspace/CasualMarket",
                "casual-market-mcp",
            ],
        }, 
        name="casual_market_mcp", 
        client_session_timeout_seconds=120  # 增加超時時間
    )
    
    # 初始化 mcp-memory-libsql
    mcp_memory = MCPServerStdio(
        params={
            "command": "npx",
            "args": ["-y", "mcp-memory-libsql"],
            "env": {"LIBSQL_URL": "file:./labs/fundamental_agent.db"},
        },
        name="mcp_memory",
        client_session_timeout_seconds=120,  # 增加超時時間
    )
    
    # 連接 servers
    await casual_market_mcp.__aenter__()
    await mcp_memory.__aenter__()
    
    mcp_servers = [casual_market_mcp, mcp_memory]
    print('✓ MCP Servers 已連接')

await setup_mcp_servers()

✓ MCP Servers 已連接


## Step 5: 初始化 Fundamental Agent

In [6]:
# 創建基本面分析 Agent
fundamental_agent = await get_fundamental_agent(
    model_name="gpt-4o-mini",
    mcp_servers=mcp_servers,
    openai_tools=openai_tools
)

print('✓ Fundamental Agent 已初始化')

2025-10-22 09:42:22 | INFO     | trading.tools.fundamental_agent:get_fundamental_agent:581 | get_fundamental_agent() called with model=gpt-4o-mini
2025-10-22 09:42:22 | INFO     | trading.tools.fundamental_agent:get_fundamental_agent:596 | Creating Agent with model=gpt-4o-mini, mcp_servers=2, tools=7
2025-10-22 09:42:22 | INFO     | trading.tools.fundamental_agent:get_fundamental_agent:611 | Fundamental Analyst Agent created successfully


✓ Fundamental Agent 已初始化


## 測試 1: 單一公司基本面分析

In [7]:
async def test_single_company_analysis():
    print('\n' + '='*70)
    print('測試 1: 單一公司基本面分析 - 台積電 (2330)')
    print('='*70)
    
    prompt = """請分析台積電 (2330) 的基本面。
    
最新財務數據：
- 營收：新台幣 250 億元
- 淨利：新台幣 50 億元
- 總資產：新台幣 1,200 億元
- 股東權益：新台幣 600 億元
- 市值：新台幣 10,000 億元
- 目前股價：新台幣 450 元

請進行以下分析：
1. 計算主要財務比率 (ROE, ROA, 淨利率)
2. 評估財務體質健全度
3. 評估當前估值水準
4. 分析成長潛力
5. 最後給出投資評級和建議

最後將結論存入 mcp_memory ，以便未來查詢和學習。
"""
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="fundamental_agent_lab", trace_id=trace_id):
            result = await Runner.run(fundamental_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None

result_1 = await test_single_company_analysis()


測試 1: 單一公司基本面分析 - 台積電 (2330)

用戶提示:
請分析台積電 (2330) 的基本面。

最新財務數據：
- 營收：新台幣 250 億元
- 淨利：新台幣 50 億元
- 總資產：新台幣 1,200 億元
- 股東權益：新台幣 600 億元
- 市值：新台幣 10,000 億元
- 目前股價：新台幣 450 元

請進行以下分析：
1. 計算主要財務比率 (ROE, ROA, 淨利率)
2. 評估財務體質健全度
3. 評估當前估值水準
4. 分析成長潛力
5. 最後給出投資評級和建議

最後將結論存入 mcp_memory ，以便未來查詢和學習。


→ Agent 執行分析...
----------------------------------------------------------------------


2025-10-22 09:42:35 | INFO     | trading.tools.fundamental_agent:calculate_financial_ratios:228 | 開始計算財務比率 | 股票: 2330
2025-10-22 09:42:35 | INFO     | trading.tools.fundamental_agent:calculate_financial_ratios:274 | 財務比率計算完成 | 股票: 2330
2025-10-22 09:42:42 | INFO     | trading.tools.fundamental_agent:analyze_financial_health:292 | 開始分析財務體質 | 股票: 2330
2025-10-22 09:42:42 | INFO     | trading.tools.fundamental_agent:analyze_financial_health:346 | 財務體質分析完成 | 股票: 2330 | 評級: C | 得分: 50 | 優勢: 2 | 弱點: 2
2025-10-22 09:42:49 | INFO     | trading.tools.fundamental_agent:evaluate_valuation:379 | 開始評估估值 | 股票: 2330 | 當前價: 450.0
2025-10-22 09:42:49 | INFO     | trading.tools.fundamental_agent:evaluate_valuation:416 | 估值評估完成 | 股票: 2330 | 等級: 昂貴 | 合理價: 382.50 | 折溢價率: -15.0%
2025-10-22 09:42:55 | INFO     | trading.tools.fundamental_agent:analyze_growth_potential:446 | 開始分析成長潛力 | 股票: 2330
2025-10-22 09:42:55 | INFO     | trading.tools.fundamental_agent:analyze_growth_potential:480 | 成長潛力分析完成 | 股票: 2330 

----------------------------------------------------------------------

Agent 分析結果:
### 台積電 (2330) 基本面分析報告

1. **主要財務比率**
   - **ROE (股東權益報酬率):** 8.33%
   - **ROA (總資產報酬率):** 4.17%
   - **淨利率:** 20%

2. **財務體質健全度**
   - **健康度評分:** 50
   - **健康評等:** C
   - **優勢:**
     - 優異的 ROE
     - 高淨利率
   - **劣勢:**
     - 高負債比 (50%)
     - 流動比率偏低 (0.00)
   - **評估:** 財務體質普通，需注意負債風險。

3. **估值水準**
   - **當前股價:** 新台幣450元
   - **合理價:** 新台幣382.5元
   - **估值評級:** 昂貴 (高估)
   - **PE Ratio:** 200 (高估)
   - **PB Ratio:** 16.67 (高估)

4. **成長潛力**
   - **營收增長:** 0%
   - **EPS 增長:** 0%
   - **增長趨勢:** 趨緩
   - **評估:** 負成長，需謹慎觀察。

5. **投資評級與建議**
   - **投資評級:** 賣出
   - **目標價:** 新台幣382.5元
   - **信心評估:** 75%
   - **建議:** 基本面轉弱，建議減碼持股。

### 關鍵優勢
- 優異的 ROE 與高淨利率顯示出其獲利能力。

### 風險提示
- 高負債比和流動比率低可能影響償債能力，需密切關注市場變化。

此分析已存入資料庫，方便未來查詢和學習。


## 測試 2: 完整 Agent 工作流程 - 多公司對比

In [ ]:
async def test_multi_company_comparison():
    print('\n' + '='*70)
    print('測試 2: 完整 Agent 工作流程 - 多公司對比分析')
    print('='*70)
    
    prompt = '''比較以下三家科技公司的基本面：
    
公司 A (2330)：
- ROE: 25%
- PE 比率: 20 倍
- 淨利潤率: 18%
- 營收成長: 15%
- EPS 成長: 18%

公司 B (2454)：
- ROE: 18%
- PE 比率: 25 倍
- 淨利潤率: 14%
- 營收成長: 22%
- EPS 成長: 28%

公司 C (3680)：
- ROE: 15%
- PE 比率: 35 倍
- 淨利潤率: 10%
- 營收成長: 8%
- EPS 成長: 5%

請分析：
1. 各公司的優劣勢
2. 估值的相對吸引力
3. 成長潛力排名
4. 投資建議排序
5. 風險評估'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="fundamental_agent_lab", trace_id=trace_id):
            result = await Runner.run(fundamental_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None

result_2 = await test_multi_company_comparison()

## 測試 3: 完整 Agent 工作流程 - 綜合基本面評估

In [ ]:
async def test_comprehensive_fundamental_analysis():
    print('\n' + '='*70)
    print('測試 3: 完整 Agent 工作流程 - 綜合基本面評估')
    print('='*70)
    
    prompt = '''進行完整的公司基本面綜合評估。
    
公司背景：
- 產業：半導體
- 市場地位：全球領導者
- 營運歷史：20 年以上

財務狀況：
- 三年平均 ROE: 22%
- 淨債務比：0.2
- 現金流狀況：強勁
- 股利政策：持續分配

市場評價：
- 當前 PE 比率：18 倍
- 產業平均 PE：20 倍
- 過去 1 年股價表現：+30%
- 技術分析：上升趨勢

發展前景：
- AI 芯片需求激增
- 產能擴充計畫
- 海外市場拓展

請進行全面的基本面評估，結合所有因素給出綜合投資評級和長期持有建議。'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行綜合分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="fundamental_agent_lab", trace_id=trace_id):
            result = await Runner.run(fundamental_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None
    
    return result

result_3 = await test_comprehensive_fundamental_analysis()

## 清理資源

In [8]:
async def cleanup_mcp_servers():
    """清理 MCP servers 連接"""
    global casual_market_mcp, mcp_memory
    
    try:
        if casual_market_mcp:
            await casual_market_mcp.__aexit__(None, None, None)
            print('✓ casual_market_mcp 已關閉')
        
        if mcp_memory:
            await mcp_memory.__aexit__(None, None, None)
            print('✓ mcp_memory 已關閉')
            
        print('✓ 所有 MCP servers 已清理完成')
        
    except Exception as e:
        print(f'清理過程中發生錯誤: {e}')

# 測試完成後執行清理
await cleanup_mcp_servers()

2025-10-22 09:45:17 | ERROR    | agents.mcp.server:cleanup:347 | Error cleaning up server: Attempted to exit cancel scope in a different task than it was entered in
2025-10-22 09:45:17 | ERROR    | agents.mcp.server:cleanup:347 | Error cleaning up server: Attempted to exit cancel scope in a different task than it was entered in


✓ casual_market_mcp 已關閉
✓ mcp_memory 已關閉
✓ 所有 MCP servers 已清理完成
